In [20]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from uszipcode import SearchEngine

In [80]:
gaz = pd.read_csv('2020_Gaz_zcta_national.txt', delimiter='\t', index_col=0)
gaz.head()
gaz.to_csv('2020_Gaz_zcta_national.csv', index=False)

In [ ]:
cols = ['geo_id', 'total_pop', 'median_income', 'PopDensity_SQMI']
to_drop = ['ALAND', 'AWATER', 'ALAND_SQMI', 'AWATER_SQMI', 'INTPTLAT', 'INTPTLONG']


In [35]:
census_2014 = pd.read_csv('./data/zip_codes_2014_5yr.csv', index_col=0).reset_index()
census_2014.head()

cols = ['geo_id', 'total_pop', 'median_income']
merge_2014 = pd.merge(census_2014[cols], gaz['ALAND_SQMI'], left_on="geo_id", right_on="GEOID", how='left')
merge_2014['PopDensity_SQMI'] = merge_2014['total_pop']/merge_2014['ALAND_SQMI']
merge_2014.head()

merge_2014[merge_2014['geo_id']==10025]

,geo_id,total_pop,median_income,ALAND_SQMI,PopDensity_SQMI
27712,10025,97373,71865.0,0.752,129485.37234


In [42]:
census_2018 = pd.read_csv('./data/zip_codes_2014_5yr.csv', index_col=0).reset_index()
census_2018.head()

cols = ['geo_id', 'total_pop', 'median_income']
merge_2018 = pd.merge(census_2018[cols], gaz['ALAND_SQMI'], left_on="geo_id", right_on="GEOID", how='left')
merge_2018['PopDensity_SQMI'] = merge_2018['total_pop']/merge_2018['ALAND_SQMI']
merge_2018.head()

merge_2018[merge_2018['geo_id']==10025]

,geo_id,total_pop,median_income,ALAND_SQMI,PopDensity_SQMI
27712,10025,97373,71865.0,0.752,129485.37234


In [37]:
merge_2018.shape

(33120, 5)

In [38]:
merge_2018.to_csv('pop_density_2018.csv', index=False)

In [40]:
zri_raw = pd.read_csv('Zip_Zri_AllHomesPlusMultifamily.csv', index_col=0)
zri_raw.head()

,RegionName,City,State,Metro,CountyName,SizeRank,2010-09,2010-10,2010-11,2010-12,...,2019-04,2019-05,2019-06,2019-07,2019-08,2019-09,2019-10,2019-11,2019-12,2020-01
RegionID,,,,,,,,,,,,,,,,,,,,,
61639,10025,New York,NY,New York-Newark-Jersey City,New York County,1,3031.0,3058.0,3031.0,3015.0,...,3785.0,3788.0,3786.0,3784.0,3766.0,3779.0,3843.0,3873.0,3835.0,NaN
84654,60657,Chicago,IL,Chicago-Naperville-Elgin,Cook County,2,1790.0,1787.0,1784.0,1774.0,...,2039.0,2070.0,2105.0,2140.0,2168.0,2185.0,2125.0,NaN,2053.0,1996.0
61637,10023,New York,NY,New York-Newark-Jersey City,New York County,3,3269.0,3304.0,3320.0,3322.0,...,3874.0,3898.0,3917.0,3929.0,3931.0,3963.0,NaN,4123.0,4079.0,NaN
91982,77494,Katy,TX,Houston-The Woodlands-Sugar Land,Harris County,4,1547.0,1549.0,1560.0,1560.0,...,1765.0,1755.0,1751.0,1752.0,1754.0,1759.0,1764.0,1769.0,1776.0,1778.0
84616,60614,Chicago,IL,Chicago-Naperville-Elgin,Cook County,5,1922.0,1925.0,1921.0,1908.0,...,2245.0,2289.0,2332.0,2372.0,2398.0,2412.0,2348.0,2356.0,2311.0,NaN


In [41]:
zri_zc_county = zri_raw[['RegionName', 'CountyName']]

In [63]:
county_2018 = pd.merge(merge_2018, zri_raw[['RegionName', 'CountyName']], left_on="geo_id", right_on="RegionName", how='left')
county_2018.head()

,geo_id,total_pop,median_income,ALAND_SQMI,PopDensity_SQMI,RegionName,CountyName
0,20899,131,20990.0,0.079,1658.227848,NaN,NaN
1,70803,1010,21346.0,0.241,4190.871369,NaN,NaN
2,40041,277,16875.0,0.104,2663.461538,NaN,NaN
3,95646,98,39375.0,3.836,25.547445,NaN,NaN
4,61112,100,24554.0,0.781,128.040973,NaN,NaN


In [64]:
county_2018.shape

(33120, 7)

In [65]:
county_2018[county_2018['RegionName'].isnull()].shape

(19958, 7)

In [66]:
search = SearchEngine(simple_zipcode=True)
county_2018['County'] = county_2018['geo_id'].apply(lambda x: search.by_zipcode(x).to_dict().get('county'))
# zori['State'] = zori['RegionName'].apply(lambda x: search.by_zipcode(x).to_dict().get('state'))
# zori['Pop'] = zori['RegionName'].apply(lambda x: search.by_zipcode(x).to_dict().get('population'))
# zori.head()

In [67]:
county_2018.head()

,geo_id,total_pop,median_income,ALAND_SQMI,PopDensity_SQMI,RegionName,CountyName,County
0,20899,131,20990.0,0.079,1658.227848,NaN,NaN,Montgomery County
1,70803,1010,21346.0,0.241,4190.871369,NaN,NaN,East Baton Rouge Parish
2,40041,277,16875.0,0.104,2663.461538,NaN,NaN,Jefferson County
3,95646,98,39375.0,3.836,25.547445,NaN,NaN,Alpine County
4,61112,100,24554.0,0.781,128.040973,NaN,NaN,Winnebago County


In [69]:
county_2018[(county_2018['CountyName']!=county_2018['County'])&(county_2018['RegionName'].notnull())].shape

(603, 8)

In [81]:
county_2018['CountyName'] = np.where(county_2018['CountyName'] == np.nan,
                                     county_2018['CountyName'], county_2018['County'])
county_2018.head()

,geo_id,total_pop,median_income,ALAND_SQMI,PopDensity_SQMI,RegionName,CountyName,County
0,20899,131,20990.0,0.079,1658.227848,NaN,Montgomery County,Montgomery County
1,70803,1010,21346.0,0.241,4190.871369,NaN,East Baton Rouge Parish,East Baton Rouge Parish
2,40041,277,16875.0,0.104,2663.461538,NaN,Jefferson County,Jefferson County
3,95646,98,39375.0,3.836,25.547445,NaN,Alpine County,Alpine County
4,61112,100,24554.0,0.781,128.040973,NaN,Winnebago County,Winnebago County


In [83]:
county_2018.drop(['RegionName', 'County'],axis=1, inplace=True)

In [85]:
county_2018.to_csv('county_pops_2018.csv', index=False)